In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import grid
from itertools import product

In [2]:
pois=pd.read_csv("pois.csv",sep=" ")

In [3]:
pois["poi_id"].duplicated().any()

False

In [4]:
# Client-side protection
def defense(lat,lon,param=1):
    """param is the penalization exponent for the utility"""
    # Get the current indices that the user is in
    i,j=grid.location_to_cell_indices(lat,lon)
    # Get all possible indices of the grids
    all_indices=product(range(1,grid.CELL_NUM_LAT+1),range(1,grid.CELL_NUM_LON+1))
    # Get the Euclidian distance of the current location to other indices
    distance=np.zeros(100)
    for index,item in enumerate(all_indices):
        distance[index]=np.sqrt((item[0]-i)**2 + (item[1]-j)**2)
    # Define the utility as 1/(1+distance)**param to avoid dividing by zero
    utility=1/(1+distance)**param
    # Normalize the probabilities
    probabilities=utility/np.sum(utility)
    # Choose the indices w.r.t. these probabilities
    choice=np.random.choice(np.arange(100),p=probabilities)
    # Get the private indices after applying the defense mechanism
    i_private,j_private=choice//10+1,choice%10+1
    # Return the representative coordinates for the grid. This is what the client will request from the server.
    # print(i,j)
    # print(i_private,j_private)
    # print(grid.location_to_cell_representative(lat,lon))
    return grid.index_to_cell_representative(i_private-1,j_private-1)

defense(46.568819955744104,6.64294176710424,param=10)

(46.566500000000005, 6.645)